<a href="https://colab.research.google.com/github/AhmedMM1122/Insect-task/blob/main/Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import Libraries
import numpy as np
from sklearn.decomposition import PCA
from scipy.stats import zscore
import pandas as pd
import time
import scipy.special
from scipy.special import gamma, gammaln
from scipy.stats import mode
from scipy.stats import mode
from sklearn.preprocessing import StandardScaler,normalize
from sklearn.decomposition import PCA
import scipy.io
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from scipy.linalg import block_diag
import argparse

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#load dataset
dataset = '/content/drive/MyDrive/INSECTS' #change this donia

fname1 = f'/content/drive/MyDrive/INSECTS/data.mat' #change this donia
fname2 = f'/content/drive/MyDrive/INSECTS/splits.mat'#change this donia

data = scipy.io.loadmat(fname1)
splits = scipy.io.loadmat(fname2)

In [4]:
embeddings_dna = data['embeddings_dna']
embeddings_img = data['embeddings_img']
labels = data['labels']
trainval_loc = splits['trainval_loc']
train_loc = splits['train_loc']
test_seen_loc = splits['test_seen_loc']
test_unseen_loc = splits['test_unseen_loc']
val_seen_loc = splits['val_seen_loc']
val_unseen_loc = splits['val_unseen_loc']
G=data['G']

In [5]:
model = 'OSBC_DNA'
transductive = True

In [6]:
#Ridge_regression

def ridge_regression(embeddings_dna, embeddings_img, data_subset, lambda_,va_l1,va_l2):
    di = embeddings_img.shape[1]
    st = data_subset
    X = embeddings_img[st[va_l1], :].T
    D = embeddings_dna[st[va_l2], :].T

    DXT = np.dot(D, X.T)
    V = np.dot(DXT, np.linalg.inv(np.dot(X, X.T) + rho * np.eye(di)))

    return V

In [7]:
if transductive:
  dd = embeddings_dna.shape[1]
  di = embeddings_img.shape[1]
  rho = 1
  if model == 'OSBC_IMG':
    print("Transductive model works well if the mapping is from Image to DNA,")
    print("Thus we automatically run this version!")
  model = 'OSBC_DIT'
  st = np.concatenate((trainval_loc, test_unseen_loc, test_seen_loc), axis=1)
  embeddings_dna = zscore(embeddings_dna, axis=0)
  embeddings_img = zscore(embeddings_img, axis=0)
  st = np.array(st).flatten()
  valid_indices = (st >= 0) & (st < embeddings_img.shape[0])
  valid_indices1 = (st >= 0) & (st < embeddings_dna.shape[0])
  start_time = time.time()
  V = ridge_regression(embeddings_dna, embeddings_img, st, rho,valid_indices,valid_indices1)
  end_time = time.time()
  elapsed_time = end_time - start_time
  print('Time took for learning map in transductive setup:', elapsed_time)

Time took for learning map in transductive setup: 12.343558073043823


In [8]:
pca_dim = 500

In [9]:
def load_tuned_params(model):
 #We use the upper() method to convert the input model string to uppercase
    model = model.upper()
 #We define the parameter sets OSBC_IMG, OSBC_DNA, OSBC_DIL, and OSBC_DIT as lists.
    OSBC_IMG = [0.1, 10, 5 * pca_dim, 1]
    OSBC_DNA = [0.1, 10, 25 * pca_dim, 0.5]
    OSBC_DIL = [0.1, 10, 5 * pca_dim, 0.5]
    OSBC_DIT = [0.1, 10, 25 * pca_dim, 0.5]
#evluate model of data, if it is img,dna,.......
    if model == 'OSBC_IMG':
        data = OSBC_IMG
    elif model == 'OSBC_DNA':
        data = OSBC_DNA
    elif model == 'OSBC_DIL':
        data = OSBC_DIL
    elif model == 'OSBC_DIT':
        data = OSBC_DIT
    else:
        raise ValueError("Invalid model name")
#The selected parameters are unpacked into k_0, k_1, m, and s, and these values are returned as a tuple
    k_0, k_1, m, s = data
    return k_0, k_1, m, s


In [10]:
k_0, k_1, m, s = load_tuned_params(model)

In [11]:
tuning = False

In [12]:
def data_split(embeddings_dna, embeddings_img, labels, trainval_loc, train_loc, test_seen_loc, test_unseen_loc, val_seen_loc, val_unseen_loc, split_type, model):
  if model == 'OSBC_IMG':
    features = embeddings_img
  else:
    features = embeddings_dna

  if split_type == 'tuning':
    train_idx = train_loc
    test_seen_idx = val_seen_loc
    test_unseen_idx = val_unseen_loc
  else:
    train_idx = trainval_loc
    test_seen_idx = test_seen_loc
    test_unseen_idx = test_unseen_loc

# Training data and labels
  xtrain = features[train_idx, :]
  ytrain = labels[train_idx]

# Test data and labels, Seen and Unseen
  valid_test_seen_idx = np.where(test_seen_idx < features.shape[0])
  valid_test_seen_idx = valid_test_seen_idx[0]  # Access the first element of the tuple
  xtest_seen = features[valid_test_seen_idx, :]
  ytest_seen = labels[valid_test_seen_idx]

  xtest_unseen = features[test_unseen_idx, :]
  ytest_unseen = labels[test_unseen_idx]

  return xtrain, ytrain, xtest_unseen, ytest_unseen, xtest_seen, ytest_seen


In [13]:
x_tr, y_tr, x_ts_us, y_ts_us, x_ts_s, y_ts_s = data_split(embeddings_dna, embeddings_img, labels, trainval_loc, train_loc, test_seen_loc,
                                                          test_unseen_loc, val_seen_loc, val_unseen_loc, 'test', model);

In [14]:
x_tr = x_tr.reshape(-1, 500)  # Reshape to (19420, 500)
y_tr = y_tr.reshape(-1, 19420)  # Reshape to (19420, 1)


In [15]:
y_tr = y_tr.reshape(-1, 1)  # Reshape to (19420, 1)

In [16]:
x_ts_us = x_ts_us.reshape(-1, 500)  # Reshape to (8463, 500)
y_ts_us = y_ts_us.reshape(-1, 8463)  # Reshape to (8463, 1)

In [17]:
y_ts_us = y_ts_us.reshape(-1, 1)  # Reshape to (8463, 1)

In [ ]:
#function of  calculates the prior mean and prior covarince in advance to feed in Bayesian classifier
import numpy as np
import pandas as pd
#takes Data (the data matrix) and labels (class labels) as input arguments.
def calculate_priors(Data, labels):
    _, dim = Data.shape
    unique_labels = np.unique(labels)
    num_classes = len(unique_labels)

    scatters = np.zeros((dim, dim, num_classes))
    class_means = np.zeros((num_classes, dim))

    for j in range(num_classes):
        class_data = Data[labels == unique_labels[j]]
#We compute the covariance matrices for each class using np.cov.
#The rowvar=False argument specifies that each column represents a variable
        scatters[:, :, j] = np.cov(class_data, rowvar=False)
        class_means[j, :] = np.mean(class_data, axis=0)
#we calculate the overall scatter matrix as the mean of the per-class scatter matrices and the overall class mean as the mean of the class means.
    scatter = np.mean(scatters, axis=2)
    mu_0 = np.mean(class_means, axis=0)

    return mu_0, scatter



In [ ]:
Data = np.random.rand(100, 5)  # Example data with 100 samples and 5 features
labels = np.random.randint(0, 3, 100)  # Example labels (0, 1, or 2)

# Call the calculate_priors function with your data and labels
mu_0, scatter = calculate_priors(Data, labels)

# Print the calculated priors and scatter matrix
print("mu_0 (class means):")
print(mu_0)
print("\nScatter matrix:")
print(scatter)

mu_0 (class means):
[0.50881256 0.49764941 0.47759555 0.43993443 0.49359455]

Scatter matrix:
[[ 0.08726868  0.01386446 -0.00262738  0.00605541  0.0046722 ]
 [ 0.01386446  0.09112498  0.01288512  0.01279908 -0.00125045]
 [-0.00262738  0.01288512  0.09956055  0.00506373  0.00422888]
 [ 0.00605541  0.01279908  0.00506373  0.0832308  -0.00395807]
 [ 0.0046722  -0.00125045  0.00422888 -0.00395807  0.0848382 ]]


In [18]:
import time
if transductive:
  #pca= 0 ,We use np.dot for matrix multiplication.
    x_tr_g = np.dot(V, embeddings_img[trainval_loc[0], :].T)
    x_tr_g = x_tr_g.T
    #we concatenate arrays using np.concatenate.
    x_tr = np.concatenate((x_tr, x_tr_g), axis=0)
    y_tr = np.concatenate((y_tr, y_tr))

# PCA for dimensionality reduction
pca_dim = 0
start_time = time.time()
C = np.cov(x_tr, rowvar=False)
#computes its eigenvalues and eigenvectors (vv)
_, vv = np.linalg.eig(C)
x_tr = np.dot(x_tr, vv[:, -pca_dim:])
x_ts_s = np.dot(x_ts_s, vv[:, -pca_dim:])
x_ts_us = np.dot(x_ts_us, vv[:, -pca_dim:])
end_time = time.time()  # Record the end time

# Calculate and print the time taken for PCA
pca_time = end_time - start_time
print(f"Time for PCA: {pca_time} seconds")
print (embeddings_img[trainval_loc, :].shape)

Time for PCA: 2.914390802383423 seconds
(1, 19420, 2048)


In [19]:
import argparse

def hyperparameter_setting(params):
    parser = argparse.ArgumentParser(description="Parse hyperparameters")
    dim = 500

    # Default values for hyperparameters
    default_k0 = 0.01
    default_k1 = 10
    default_m = 5 * dim
    default_mu0 = 0
    default_s = 1
    default_iter = 1
    default_pca = 0
    default_scatter = 0
    default_tuning = False

    parser.add_argument('--kappa_0', type=float, default=default_k0)
    parser.add_argument('--kappa_1', type=float, default=default_k1)
    parser.add_argument('--cov_shape', type=float, default=default_m)
    parser.add_argument('--prior_mean', type=float, default=default_mu0)
    parser.add_argument('--prior_covscale', type=float, default=default_s)
    parser.add_argument('--iter', type=int, default=default_iter)
    parser.add_argument('--pca', type=int, default=default_pca)
    parser.add_argument('--scatter', type=int, default=default_scatter)
    parser.add_argument('--tuning', type=bool, default=default_tuning)

    args = parser.parse_args(params)

    k_0 = args.kappa_0
    k_1 = args.kappa_1
    m = args.cov_shape
    num_iter = args.iter
    pca_dim = args.pca
    mu_0 = args.prior_mean
    s = args.prior_covscale
    scatter = args.scatter
    tuning = args.tuning

    if mu_0 == 0:
        mu_0 = [0] * dim
    elif scatter == 0:
        scatter = [[1 if i == j else 0 for j in range(dim)] for i in range(dim)]

    return k_0, k_1, m, mu_0, s, scatter, num_iter, pca_dim, tuning

In [20]:
parameters = {
    'kappa_0': k_0,
    'kappa_1': k_1,
    'cov_shape': m,
    'prior_covscale': s,
    'pca': pca_dim
    # Add other parameters as needed
}

In [21]:
arg_list = [f'--{key} {value}' for key, value in parameters.items()]

In [22]:
arg_list = []
for key, value in parameters.items():
    arg_list.extend(['--' + key, str(value)])


In [23]:
k_0, k_1, m, mu_0, s, scatter, num_iter, pca_dim, tuning = hyperparameter_setting(arg_list)

In [24]:
import numpy as np
#The ppd_derivation function appears to calculate class predictive statistics (such as covariances, means, and degrees of freedom)
# for a dataset with class labels and genus labels
def ppd_derivation(X, Y, G, Psi, mu0, m, k0, k1): #X -> The dataset, where each row represents a data point, and each column represents a feature.
#Y -> Class labels for each data point in X.
#G ->enus labels for each data point in X.
#Psi -> Initial covariance matrix.
#mu0 -> Initial mean vector.
#m, k0, k1 -> Hyperparameters used in the calculations.
    seenclasses = np.unique(Y)
#Separates seen classes (those with class labels) from surrogate classes (those without class labels) based on the provided genus labels.
    seengenera = G[seenclasses]
    uyg = np.unique(G)
    ct = len(uyg)
    nc = len(seenclasses) + ct
    n, d = X.shape
#Sig_s -> Class predictive covariances for each class (seen and surrogate).
#mu_s -> Class predictive means for each class.
#v_s -> Class predictive degrees of freedom for each class.
#class_id -> Class IDs indicating which class each set of statistics belongs to.
#Sigmas -> Intermediate results for covariance calculations.
    Sig_s = np.zeros((d, d, nc))
    Sigmas = np.zeros((d, d, nc))
    mu_s = np.zeros((nc, d))
    v_s = np.zeros(nc)

    uy = seenclasses
    ncl = len(uy)
    cnt = 0
#For each seen class:

#1-Computes class-specific statistics, including the number of points, mean, and scatter matrix.
#2-Calculates class predictive statistics, taking into account data likelihood and global prior.
    for i in range(ncl):
        in_idx = Y == uy[i]
        Xi = X[in_idx, :]

        cur_n = np.sum(in_idx)
        cur_S = (cur_n - 1) * np.cov(Xi, rowvar=False)
        cur_mu = np.mean(Xi, axis=0)

        v_s[cnt] = cur_n + m - d + 1
        mu_s[cnt, :] = (cur_n * cur_mu + (k0 * k1 / (k0 + k1)) * mu0) / (cur_n + (k0 * k1 / (k0 + k1)))
        Smu = ((cur_n * (k0 * k1 / (k0 + k1))) / ((k0 * k1 / (k0 + k1)) + cur_n)) * np.outer(cur_mu - mu0, cur_mu - mu0)
        Sig_s[:, :, cnt] = (Psi + cur_S + Smu) / (((cur_n + (k0 * k1 / (k0 + k1))) * v_s[cnt]) / (cur_n + (k0 * k1 / (k0 + k1)) + 1))
        cnt += 1

    uy = np.unique(G)
    ncl = len(uy)
    class_id = np.zeros(nc)
    cnt = 0

    for i in range(ncl):
        in_idx = np.zeros(n, dtype=bool)

        genus = uy[i]
        classes = seenclasses[seengenera == genus]
        nci = len(classes)

        if nci >= 1:
#For each surrogate genus class:

#Identifies the classes belonging to the genus.
#Calculates component-specific statistics (means, scatter matrices, etc.) for the classes within the genus.
#Computes genus-specific predictive statistics based on the component statistics.
            for j in range(nci):
                in_idx[Y == classes[j]] = True

            Yi = Y[in_idx]
            Xi = X[in_idx, :]
            uyi = np.unique(Yi)

            ncpi = len(uyi)
            xkl = np.zeros((ncpi, d))
            Skl = np.zeros((d, d, ncpi))
            kap = np.zeros(ncpi)
            nkl = np.zeros(ncpi)

            for j in range(ncpi):
                in_idx = Yi == uyi[j]
                nkl[j] = np.sum(in_idx)
                kap[j] = nkl[j] * k1 / (nkl[j] + k1)
                Xij = Xi[in_idx, :]
                xkl[j, :] = np.mean(Xij, axis=0)
                Skl[:, :, j] = (nkl[j] - 1) * np.cov(Xij, rowvar=False)
#Collects the computed class predictive statistics, class IDs, and intermediate results for all classes (both seen and surrogate)
            sumkap = np.sum(kap)
            kaps = (sumkap + k0) * k1 / (sumkap + k0 + k1)
            sumSkl = np.sum(Skl, axis=2)
            muk = (np.sum(xkl * (kap.reshape(-1, 1)), axis=0) + k0 * mu0) / (np.sum(kap) + k0)
            vsc = np.sum(nkl) - ncpi + m - d + 1
            class_id[cnt] = uy[i]
            v_s[cnt] = vsc
            Sigmas[:, :, cnt] = Psi + sumSkl
            Sig_s[:, :, cnt] = (Psi + sumSkl) / ((kaps * v_s[cnt]) / (kaps + 1))
            mu_s[cnt, :] = muk
            cnt += 1

    return Sig_s, mu_s, v_s, class_id, Sigmas


In [58]:
def predict(X, Sig_s, mu_s, v_s, class_id):
    ncl, d = mu_s.shape
    piconst = (d / 2) * np.log(np.pi)
    gl_pc = scipy.special.gammaln(0.5 + np.arange(0.5, max(v_s) + d, 0.5))
    n = X.shape[0]
    prob_mat = np.zeros((n, ncl))

    # Calculating log student-t likelihood for numerical stability
    for j in range(ncl):
        v = X - mu_s[j, :]  # Center the data
        chsig = np.linalg.cholesky(Sig_s[:, :, j])  # Cholesky decomposition
        tpar = gl_pc[v_s[j] + d] - (gl_pc[v_s[j]] + (d / 2) * np.log(v_s[j]) + piconst) - np.sum(np.log(np.diag(chsig)))  # Stu-t lik part 1
        temp = np.linalg.solve(chsig, v.T).T
        prob_mat[:, j] = tpar - 0.5 * (v_s[j] + d) * np.log(1 + (1 / v_s[j]) * np.sum(temp * temp, axis=1))

    bb = np.argmax(prob_mat, axis=1)
    ypred = class_id[bb]  # To ensure labels are correctly assigned back to the original ones

    return ypred, prob_mat

In [61]:

# Initialize example values for X, Sig_s, mu_s, v_s, and class_id
X = np.array([[1.0, 2.0], [2.0, 3.0], [3.0, 4.0]])  # Example data
Sig_s = np.array([[[1.0, 0.0], [0.0, 1.0]], [[1.0, 0.0], [0.0, 1.0]]])  # Example covariance matrices
mu_s = np.array([[1.0, 2.0], [3.0, 4.0]])  # Example means
v_s = np.array([2.0, 3.0])  # Example v_s values
class_id = np.array([0, 1])  # Example class IDs



# Call the predict1 function with the example values
ypred_unseen, prob_mat_us = predict(X, Sig_s, mu_s, v_s, class_id)

# Print the results
print("Predicted Class Labels:", ypred_unseen)
print("Probability Matrix:")
print(prob_mat_us)


LinAlgError: ignored